# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [2]:
import pandas as pd 
import numpy as np 
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import pandas as pd 
import numpy as np 
from nltk import word_tokenize
import nltk
nltk.download('punkt')
data=pd.read_csv('preprocessed_descriptions.csv')
words= set()
for i in range(len(data)):
  w=[]
  if type (data.preprocessed_descriptions[i])==str:
    w.append(word_tokenize(data.preprocessed_descriptions[i]))
    for j in range(len(w)):
      words.update(w[j])
words=list(words)
print(words[:5])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['portability', 'coats', 'hendon', 'clever', 'expected']


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [5]:
from nltk.metrics.distance import edit_distance
import random 
for i in range (5):
  a = random.randint(0,len(words))
  b = random.randint(0,len(words))
  c=edit_distance(words[a], words[b])
  print(f'Между "{words[a]}", и "{words[b]}", {c} шагов')

Между "commercially", и "japalenos", 12 шагов
Между "diwalishubh", и "dieniab", 7 шагов
Между "holes", и "pickin", 6 шагов
Между "unripe", и "qeth", 6 шагов
Между "winging", и "pak", 7 шагов


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [42]:
from nltk.metrics.distance import edit_distance
from nltk import word_tokenize
import pandas as pd

data = pd.read_csv('preprocessed_descriptions.csv')
words = set()

for row in data.itertuples():
    words.update(word_tokenize(row[1]))

def find_similar_words(word, k, words):
    similar_words = []
    for w in words:
        if edit_distance(word, w) <= k:
            similar_words.append(w)
        if len(similar_words) >= k:
            break
    return similar_words

find_similar_words('delicious' ,4, words)

['luscious', 'bodacious', 'delicacies', 'delicies']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [53]:
import pandas as pd 
import numpy as np 
from nltk import word_tokenize
import nltk
nltk.download('wordnet')
nltk.download('punkt')
data=pd.read_csv('preprocessed_descriptions.csv')
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
stemmer_eng= SnowballStemmer('english')
lemmatizer= WordNetLemmatizer()

lemmatizer.lemmatize("rocks")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'rock'

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [52]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
data = pd.read_csv('preprocessed_descriptions.csv')

stop_words = set(stopwords.words('english'))

total_stop_words = 0
total_words = 0
word_freq = {}

for description in data.preprocessed_descriptions:
    for word in nltk.word_tokenize(str(description)):
        total_words += 1
        if word in stop_words:
            total_stop_words += 1
            continue
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

stop_words_pro = total_stop_words / total_words
print('Stop words: {:.2%}'.format(stop_words_pro))

# Get top 10 
sorted_word = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
top10_words = sorted_word[:10]

print('Top 10 most frequent words:')
for word, freq in top10_words:
    print('{}: {}'.format(word, freq))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Stop words proportion: 45.65%
Top 10 most frequent words:
recipe: 14871
make: 6326
time: 5137
use: 4620
great: 4430
like: 4167
easy: 4152
one: 3872
made: 3810
good: 3791


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [46]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
from nltk import sent_tokenize

f = pd.read_csv('preprocessed_descriptions.csv')
random_recipes = f.sample(5)
random_recipes["preprocessed_descriptions"] = random_recipes["preprocessed_descriptions"].map(sent_tokenize)
t = TfidfVectorizer()
text = list()
for index, row in random_recipes.iterrows():
  text.append(row["preprocessed_descriptions"][0])

t2 = t.fit_transform(text)
t3 = t2.toarray()
print(t3)

[[0.         0.         0.         0.         0.         0.06737488
  0.18048698 0.         0.06737488 0.21743053 0.         0.06737488
  0.         0.         0.13474977 0.13474977 0.         0.06737488
  0.06737488 0.06737488 0.06737488 0.06737488 0.         0.
  0.06737488 0.         0.06737488 0.06737488 0.13474977 0.
  0.         0.         0.         0.         0.         0.06737488
  0.         0.         0.13474977 0.06737488 0.         0.06737488
  0.06737488 0.         0.06737488 0.06737488 0.06737488 0.06737488
  0.         0.         0.         0.         0.06737488 0.06737488
  0.04512174 0.         0.         0.         0.06737488 0.
  0.06737488 0.         0.         0.10871526 0.         0.
  0.         0.         0.         0.05435763 0.06737488 0.
  0.05435763 0.09024349 0.06737488 0.06737488 0.06737488 0.06737488
  0.         0.10871526 0.         0.06737488 0.         0.06737488
  0.13474977 0.         0.06737488 0.         0.06737488 0.10871526
  0.         0.06737

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).